In [1]:
import gym
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K
import numpy as np

In [2]:

def initiate_NN(learning_rate=1e-3):
    model = Sequential()
    model.add(Dense(64, input_dim=8, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(lr=learning_rate))
    return model

def epsilon_action(epsilon):
    random_value = np.random.random()
    if random_value<=epsilon:
        action = env.action_space.sample()
    else:
        Y=online_model.predict(state.reshape(1,-1))
        action = np.argmax(Y[0])
    return action


In [3]:


gamma=0.99
epsilon_decay = 0.002
learning_rate =0.0002
max_e=0.6
min_e=0.0001
replay_batch_size=30
iterations=5000

In [4]:
env = gym.make('LunarLander-v2')
state=env.reset()
online_model = initiate_NN(learning_rate)
target_model =initiate_NN(learning_rate)
target_model.set_weights(online_model.get_weights())
memory=[]
episode_reward_rolling = []
episode_reward=[]

for episode in range(iterations):
    state=env.reset()
    current_episode_reward = 0
    done=False
    steps=0
    epsilon = min_e + (max_e - min_e)*np.exp(-epsilon_decay*episode)
    while not done:
        action = epsilon_action(epsilon)
        new_state, reward, done, info= env.step(action)
        steps+=1
        current_episode_reward +=reward
        memory.append([state,action,reward,new_state,done])
        state = new_state
        if len(memory)>=replay_batch_size:
            batch = random.sample(memory, replay_batch_size)
            batch = np.array(batch)
            non_terminal_samples = np.where(batch[:, 4] == False)
            y = np.copy(batch[:, 2])
            if len( non_terminal_samples[0]) > 0:
                #use online q values to select best action
                online_q = online_model.predict(np.vstack(batch[:, 3]))
                #use offline q values of best action as target 
                target_q = target_model.predict(np.vstack(batch[:, 3]))
                y[non_terminal_samples] += np.multiply(gamma, \
                            target_q[non_terminal_samples, \
                            np.argmax(online_q[non_terminal_samples, :][0], axis=1)][0])
            actions = np.array(batch[:, 1], dtype=int)
            y_target = online_model.predict(np.vstack(batch[:, 0]))
            y_target[range(replay_batch_size), actions] = y
            online_model.fit(np.vstack(batch[:, 0]), y_target, epochs=1, verbose=0)
        if done:
            target_model.set_weights(online_model.get_weights())

    episode_reward.append(current_episode_reward) 
    if len(memory)>=5000:
        memory = memory[-5000:]
    if len(episode_reward)>=100:
        episode_reward_rolling.append(np.average(episode_reward[-100:]))
    else:
        episode_reward_rolling.append(np.average(episode_reward))
    print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
          ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
          '%.2f' % epsilon)

episode:  0  episode reward:  -172.85  rolling reward:  -172.85  Steps:  106  epsilon:  0.60
episode:  1  episode reward:  -354.52  rolling reward:  -263.68  Steps:  69  epsilon:  0.60
episode:  2  episode reward:  -194.22  rolling reward:  -240.53  Steps:  141  epsilon:  0.60
episode:  3  episode reward:  -45.05  rolling reward:  -191.66  Steps:  116  epsilon:  0.60
episode:  4  episode reward:  -48.61  rolling reward:  -163.05  Steps:  118  epsilon:  0.60
episode:  5  episode reward:  -149.83  rolling reward:  -160.85  Steps:  126  epsilon:  0.59
episode:  6  episode reward:  -109.39  rolling reward:  -153.50  Steps:  123  epsilon:  0.59
episode:  7  episode reward:  -428.24  rolling reward:  -187.84  Steps:  104  epsilon:  0.59
episode:  8  episode reward:  -258.23  rolling reward:  -195.66  Steps:  301  epsilon:  0.59
episode:  9  episode reward:  -132.82  rolling reward:  -189.38  Steps:  182  epsilon:  0.59
episode:  10  episode reward:  -226.79  rolling reward:  -192.78  Steps: 

episode:  88  episode reward:  -97.46  rolling reward:  -110.89  Steps:  140  epsilon:  0.50
episode:  89  episode reward:  -123.98  rolling reward:  -111.03  Steps:  133  epsilon:  0.50
episode:  90  episode reward:  -105.60  rolling reward:  -110.97  Steps:  151  epsilon:  0.50
episode:  91  episode reward:  -83.42  rolling reward:  -110.67  Steps:  200  epsilon:  0.50
episode:  92  episode reward:  -58.87  rolling reward:  -110.12  Steps:  122  epsilon:  0.50
episode:  93  episode reward:  -99.59  rolling reward:  -110.01  Steps:  114  epsilon:  0.50
episode:  94  episode reward:  -119.48  rolling reward:  -110.11  Steps:  194  epsilon:  0.50
episode:  95  episode reward:  -20.45  rolling reward:  -109.17  Steps:  133  epsilon:  0.50
episode:  96  episode reward:  -116.46  rolling reward:  -109.25  Steps:  335  epsilon:  0.50
episode:  97  episode reward:  -66.05  rolling reward:  -108.81  Steps:  258  epsilon:  0.49
episode:  98  episode reward:  -55.27  rolling reward:  -108.26  S

episode:  176  episode reward:  123.89  rolling reward:  -69.61  Steps:  1000  epsilon:  0.42
episode:  177  episode reward:  45.52  rolling reward:  -68.26  Steps:  1000  epsilon:  0.42
episode:  178  episode reward:  -78.88  rolling reward:  -67.38  Steps:  140  epsilon:  0.42
episode:  179  episode reward:  -56.85  rolling reward:  -66.87  Steps:  138  epsilon:  0.42
episode:  180  episode reward:  -228.13  rolling reward:  -68.48  Steps:  116  epsilon:  0.42
episode:  181  episode reward:  -47.23  rolling reward:  -67.81  Steps:  247  epsilon:  0.42
episode:  182  episode reward:  -87.08  rolling reward:  -67.81  Steps:  159  epsilon:  0.42
episode:  183  episode reward:  -26.57  rolling reward:  -67.07  Steps:  180  epsilon:  0.42
episode:  184  episode reward:  75.42  rolling reward:  -65.55  Steps:  1000  epsilon:  0.42
episode:  185  episode reward:  95.78  rolling reward:  -64.34  Steps:  1000  epsilon:  0.41
episode:  186  episode reward:  -61.97  rolling reward:  -64.18  Ste

episode:  265  episode reward:  153.34  rolling reward:  10.74  Steps:  1000  epsilon:  0.35
episode:  266  episode reward:  63.28  rolling reward:  10.87  Steps:  1000  epsilon:  0.35
episode:  267  episode reward:  19.05  rolling reward:  11.83  Steps:  339  epsilon:  0.35
episode:  268  episode reward:  -13.62  rolling reward:  10.90  Steps:  79  epsilon:  0.35
episode:  269  episode reward:  165.06  rolling reward:  11.56  Steps:  1000  epsilon:  0.35
episode:  270  episode reward:  19.12  rolling reward:  11.22  Steps:  133  epsilon:  0.35
episode:  271  episode reward:  27.70  rolling reward:  11.91  Steps:  137  epsilon:  0.35
episode:  272  episode reward:  125.33  rolling reward:  12.67  Steps:  1000  epsilon:  0.35
episode:  273  episode reward:  -24.68  rolling reward:  12.36  Steps:  1000  epsilon:  0.35
episode:  274  episode reward:  119.63  rolling reward:  14.09  Steps:  1000  epsilon:  0.35
episode:  275  episode reward:  -53.70  rolling reward:  16.41  Steps:  116  ep

episode:  354  episode reward:  -49.56  rolling reward:  34.88  Steps:  109  epsilon:  0.30
episode:  355  episode reward:  14.14  rolling reward:  36.52  Steps:  169  epsilon:  0.30
episode:  356  episode reward:  283.67  rolling reward:  39.44  Steps:  393  epsilon:  0.29
episode:  357  episode reward:  245.15  rolling reward:  41.79  Steps:  238  epsilon:  0.29
episode:  358  episode reward:  162.97  rolling reward:  44.05  Steps:  1000  epsilon:  0.29
episode:  359  episode reward:  86.32  rolling reward:  43.57  Steps:  1000  epsilon:  0.29
episode:  360  episode reward:  60.54  rolling reward:  43.80  Steps:  1000  epsilon:  0.29
episode:  361  episode reward:  124.59  rolling reward:  44.27  Steps:  1000  epsilon:  0.29
episode:  362  episode reward:  149.62  rolling reward:  46.02  Steps:  1000  epsilon:  0.29
episode:  363  episode reward:  263.01  rolling reward:  49.61  Steps:  968  epsilon:  0.29
episode:  364  episode reward:  98.98  rolling reward:  50.23  Steps:  1000  e

...

episode:  709  episode reward:  285.95  rolling reward:  191.36  Steps:  249  epsilon:  0.15
episode:  710  episode reward:  144.93  rolling reward:  190.74  Steps:  1000  epsilon:  0.15
episode:  711  episode reward:  144.86  rolling reward:  190.40  Steps:  1000  epsilon:  0.14
episode:  712  episode reward:  302.75  rolling reward:  190.49  Steps:  348  epsilon:  0.14
episode:  713  episode reward:  254.60  rolling reward:  193.07  Steps:  322  epsilon:  0.14
episode:  714  episode reward:  273.66  rolling reward:  194.29  Steps:  477  epsilon:  0.14
episode:  715  episode reward:  277.49  rolling reward:  197.97  Steps:  936  epsilon:  0.14
episode:  716  episode reward:  243.51  rolling reward:  199.87  Steps:  310  epsilon:  0.14
episode:  717  episode reward:  259.80  rolling reward:  199.87  Steps:  459  epsilon:  0.14
episode:  718  episode reward:  254.37  rolling reward:  200.12  Steps:  252  epsilon:  0.14
episode:  719  episode reward:  -202.90  rolling reward:  198.12  St

episode:  798  episode reward:  279.29  rolling reward:  200.59  Steps:  272  epsilon:  0.12
episode:  799  episode reward:  255.66  rolling reward:  200.45  Steps:  389  epsilon:  0.12
episode:  800  episode reward:  196.11  rolling reward:  199.89  Steps:  360  epsilon:  0.12
episode:  801  episode reward:  270.08  rolling reward:  199.81  Steps:  457  epsilon:  0.12
episode:  802  episode reward:  244.89  rolling reward:  199.62  Steps:  586  epsilon:  0.12
episode:  803  episode reward:  217.87  rolling reward:  198.94  Steps:  542  epsilon:  0.12
episode:  804  episode reward:  -192.82  rolling reward:  194.10  Steps:  373  epsilon:  0.12
episode:  805  episode reward:  271.14  rolling reward:  194.22  Steps:  220  epsilon:  0.12
episode:  806  episode reward:  278.03  rolling reward:  194.27  Steps:  726  epsilon:  0.12
episode:  807  episode reward:  253.73  rolling reward:  194.15  Steps:  451  epsilon:  0.12
episode:  808  episode reward:  -227.72  rolling reward:  189.03  Ste

episode:  887  episode reward:  215.69  rolling reward:  209.08  Steps:  652  epsilon:  0.10
episode:  888  episode reward:  243.23  rolling reward:  211.57  Steps:  420  epsilon:  0.10
episode:  889  episode reward:  231.42  rolling reward:  212.26  Steps:  359  epsilon:  0.10
episode:  890  episode reward:  224.21  rolling reward:  212.26  Steps:  552  epsilon:  0.10
episode:  891  episode reward:  241.60  rolling reward:  214.61  Steps:  420  epsilon:  0.10
episode:  892  episode reward:  272.63  rolling reward:  214.99  Steps:  316  epsilon:  0.10
episode:  893  episode reward:  260.15  rolling reward:  216.04  Steps:  277  epsilon:  0.10
episode:  894  episode reward:  237.89  rolling reward:  216.02  Steps:  596  epsilon:  0.10
episode:  895  episode reward:  184.94  rolling reward:  215.09  Steps:  351  epsilon:  0.10
episode:  896  episode reward:  237.19  rolling reward:  219.07  Steps:  241  epsilon:  0.10
episode:  897  episode reward:  212.66  rolling reward:  219.10  Steps

episode:  976  episode reward:  264.31  rolling reward:  205.46  Steps:  284  epsilon:  0.09
episode:  977  episode reward:  234.98  rolling reward:  206.02  Steps:  230  epsilon:  0.09
episode:  978  episode reward:  207.64  rolling reward:  206.03  Steps:  288  epsilon:  0.08
episode:  979  episode reward:  283.31  rolling reward:  205.94  Steps:  274  epsilon:  0.08
episode:  980  episode reward:  211.45  rolling reward:  206.25  Steps:  387  epsilon:  0.08
episode:  981  episode reward:  219.54  rolling reward:  206.36  Steps:  454  epsilon:  0.08
episode:  982  episode reward:  278.31  rolling reward:  206.19  Steps:  262  epsilon:  0.08
episode:  983  episode reward:  274.99  rolling reward:  206.66  Steps:  297  epsilon:  0.08
episode:  984  episode reward:  191.44  rolling reward:  206.44  Steps:  275  epsilon:  0.08
episode:  985  episode reward:  215.14  rolling reward:  206.21  Steps:  313  epsilon:  0.08
episode:  986  episode reward:  275.72  rolling reward:  206.73  Steps

KeyboardInterrupt: 

In [6]:
online_model.save_weights('weights_DQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)+'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.h5')

import pickle

with open('weights_DQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)+'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', 'wb') as fp:
    pickle.dump({'rewards':episode_reward,'rolling_reward':episode_reward_rolling}, fp)


In [167]:
rward = pickle.load( open( 'weights_DQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', "rb" ) )

In [168]:
rward

{'rewards': [-131.3223570990876, -248.23258991485326, -133.48636655895385],
 'rolling_reward': [-131.3223570990876,
  -189.77747350697044,
  -171.01377119096492]}

In [163]:
episode_reward

[]

In [152]:
Y_target.shape

(5, 4)

In [142]:
batch[terminal_samples].shape

(0, 5)

In [125]:



# i:index for sample [state,action,reward,new_state,done]
for i,sample in enumerate(batch):
    terminal_sample = sample[4]
    new_state_sample = sample[3]
    state_sample = sample[0]
    X[i]=state_sample
    action_sample = sample[1]
    Y_target[i] = online_model.predict(state_sample.reshape(1,-1))[0]
    if terminal:
        #terminal then targetY is reward
        Y_target[i][action_sample] = sample[2]
    else:
        #use online weights to select argmax action under new_state

        online_qvalues_batch =online_model.predict(new_state_sample.reshape(1,-1)) 
        max_action = np.argmax(online_qvalues_batch)
        #use target weights to calculate Qvalue
        #non-terminal then targetY is reward +gamma*new Qvalue
        Y_target[i][action_sample] = sample[2] + gamma * target_model.predict(new_state_sample.reshape(1,-1)).reshape(4,)[max_action]

online_model.fit(np.array(X), np.array(Y_target), verbose=0)


100

In [ ]:
Notice that the selection of the action, in the argmax, is


In [91]:
online_qvalues_batch

[array([[ 0.18740644, -0.2721934 ,  0.2233217 , -0.45678362]],
       dtype=float32),
 array([[ 0.19613695, -0.27545747,  0.20470577, -0.46681872]],
       dtype=float32),
 array([[ 0.18746991, -0.25133634,  0.19952038, -0.4359238 ]],
       dtype=float32),
 array([[ 0.19334945, -0.2700162 ,  0.19816679, -0.45707738]],
       dtype=float32),
 array([[ 0.19308917, -0.27386588,  0.21208042, -0.46272066]],
       dtype=float32)]

In [113]:
np.array(X)

array([[-0.00213823,  1.4003743 , -0.21659775, -0.4687014 ,  0.0024845 ,
         0.04906261,  0.        ,  0.        ],
       [-0.00846777,  1.3651963 , -0.20548455, -0.5481141 ,  0.00758275,
         0.00502167,  0.        ,  0.        ],
       [-0.01887293,  1.3004851 , -0.22156799, -0.61512053,  0.00625335,
         0.04479944,  0.        ,  0.        ],
       [-0.01250668,  1.3402183 , -0.20310256, -0.5355056 ,  0.00488083,
        -0.02924636,  0.        ,  0.        ],
       [-0.00641537,  1.3775289 , -0.21631236, -0.5210343 ,  0.00733169,
         0.04846335,  0.        ,  0.        ]], dtype=float32)

In [112]:
Y_target

[array([-1.2552428 , -0.2741652 ,  0.2263453 , -0.45686692], dtype=float32),
 array([ 0.19308917, -0.27386588,  0.21208042, -0.6722371 ], dtype=float32),
 array([ 0.1933487, -0.2546284,  5.1014786, -0.445849 ], dtype=float32),
 array([-0.91460276, -0.27277392,  0.20281965, -0.4571485 ], dtype=float32),
 array([ 0.18904   , -0.27001685,  0.21992686, -0.7534547 ], dtype=float32)]

In [ ]:

class DDQL:
    def __init__(self, nS, nA):
        self.nS = nS
        self.nA = nA
        self.epsilon = 1
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9993
        self.gamma = 0.99
        self.learning_rate = 0.0001
        self.epochs = 1
        self.verbose = 0
        self.minibatch_size = 30
        self.memory = deque(maxlen=5000)
        self.model = self.create_model()
        self.target_model = self.create_model()

    def create_model(self):
        model = Sequential()

        # Add 2 hidden layers with 64 nodes each
        model.add(Dense(64, input_dim=self.nS, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.nA, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def add_memory(self, s, a, r, s_prime, done):
        self.memory.append((s, a, r, s_prime, done))

    def target_model_update(self):
        self.target_model.set_weights(self.model.get_weights())

    def selectAction(self, s):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.nA)
        q = self.model.predict(s)
        return np.argmax(q[0])

    def replay(self):
        # Vectorized method for experience replay
        minibatch = random.sample(self.memory, self.minibatch_size)
        minibatch = np.array(minibatch)
        not_done_indices = np.where(minibatch[:, 4] == False)
        y = np.copy(minibatch[:, 2])

        # If minibatch contains any non-terminal states, use separate update rule for those states
        if len(not_done_indices[0]) > 0:
            predict_sprime = self.model.predict(np.vstack(minibatch[:, 3]))
            predict_sprime_target = self.target_model.predict(np.vstack(minibatch[:, 3]))
            
            # Non-terminal update rule
            y[not_done_indices] += np.multiply(self.gamma, \
                        predict_sprime_target[not_done_indices, \
                        np.argmax(predict_sprime[not_done_indices, :][0], axis=1)][0])

        actions = np.array(minibatch[:, 1], dtype=int)
        y_target = self.model.predict(np.vstack(minibatch[:, 0]))
        y_target[range(self.minibatch_size), actions] = y
        self.model.fit(np.vstack(minibatch[:, 0]), y_target, epochs=self.epochs, verbose=self.verbose)

    def replayIterative(self):
        # Iterative method - this performs the same function as replay() but is not vectorized 
        s_list = []
        y_state_list = []
        minibatch = random.sample(self.memory, self.minibatch_size)
        for s, a, r, s_prime, done in minibatch:
            s_list.append(s)
            y_action = r
            if not done:
                y_action = r + self.gamma * np.amax(self.model.predict(s_prime)[0])

            print(y_action)
            
            y_state = self.model.predict(s)
            y_state[0][a] = y_action
            y_state_list.append(y_state)
        self.model.fit(np.squeeze(s_list), np.squeeze(y_state_list), batch_size=self.minibatch_size, epochs=1, verbose=0)

if __name__ == "__main__":
    np.set_printoptions(precision=2)
    env = gym.make('LunarLander-v2')
    env = wrappers.Monitor(env, tdir, force=True, video_callable=False)

    nS = env.observation_space.shape[0]
    nA = env.action_space.n

    agent = DDQL(nS, nA)

    # Set to true to use saved model
    episodes = 10000

    # Cumulative reward
    reward_avg = deque(maxlen=100)

    for e in range(episodes):
        episode_reward = 0
        s = env.reset()
        s = np.reshape(s, [1, nS])

        for time in range(1000):

            # Query next action from learner and perform action
            a = agent.selectAction(s)
            s_prime, r, done, info = env.step(a)

            # Add cumulative reward
            episode_reward += r

            # Reshape new state
            s_prime = np.reshape(s_prime, [1, nS])

            # Add experience to memory
            agent.add_memory(s, a, r, s_prime, done)

            # Set current state to new state
            s = s_prime

            #Perform experience replay if memory length is greater than minibatch length
   
            if len(agent.memory) > agent.minibatch_size:
                agent.replay()

            # If episode is done, exit loop
            if done:
                agent.target_model_update()
                break


        # epsilon decay
        if agent.epsilon > agent.epsilon_min:
            agent.epsilon *= agent.epsilon_decay

        # Running average of past 100 episodes
        reward_avg.append(episode_reward)
        print('episode: ', e, ' score: ', '%.2f' % episode_reward, ' avg_score: ', '%.2f' % np.average(reward_avg), ' frames: ', time, ' epsilon: ', '%.2f' % agent.epsilon)
        
        # with open('trained_agent.txt', 'a') as f:
        #     f.write(str(np.average(reward_avg)) + '\n')

    env.close()
#     gym.upload(tdir, api_key='sk_EJo79Jo0RsSEA4EF6Cp5mg')


#agent.model.save_weights('weights.h5')
#agent.model.load_weights('weights.h5')

episode:  0  score:  -101.20  avg_score:  -101.20  frames:  104  epsilon:  1.00
episode:  1  score:  -147.99  avg_score:  -124.59  frames:  74  epsilon:  1.00
episode:  2  score:  -95.67  avg_score:  -114.95  frames:  74  epsilon:  1.00
episode:  3  score:  -218.30  avg_score:  -140.79  frames:  94  epsilon:  1.00
episode:  4  score:  -231.98  avg_score:  -159.03  frames:  86  epsilon:  1.00
episode:  5  score:  -161.32  avg_score:  -159.41  frames:  68  epsilon:  1.00
episode:  6  score:  -96.64  avg_score:  -150.44  frames:  66  epsilon:  1.00
episode:  7  score:  -141.43  avg_score:  -149.32  frames:  97  epsilon:  0.99
episode:  8  score:  -480.81  avg_score:  -186.15  frames:  102  epsilon:  0.99
episode:  9  score:  -371.18  avg_score:  -204.65  frames:  92  epsilon:  0.99
episode:  10  score:  -161.56  avg_score:  -200.73  frames:  95  epsilon:  0.99
episode:  11  score:  -80.20  avg_score:  -190.69  frames:  61  epsilon:  0.99
episode:  12  score:  -332.47  avg_score:  -201.60 

episode:  103  score:  -168.96  avg_score:  -179.65  frames:  118  epsilon:  0.93
episode:  104  score:  -64.43  avg_score:  -177.97  frames:  91  epsilon:  0.93
episode:  105  score:  -200.70  avg_score:  -178.36  frames:  72  epsilon:  0.93
episode:  106  score:  32.45  avg_score:  -177.07  frames:  121  epsilon:  0.93
episode:  107  score:  -260.25  avg_score:  -178.26  frames:  117  epsilon:  0.93
episode:  108  score:  -110.97  avg_score:  -174.56  frames:  80  epsilon:  0.93
episode:  109  score:  -253.15  avg_score:  -173.38  frames:  94  epsilon:  0.93
episode:  110  score:  -125.12  avg_score:  -173.02  frames:  88  epsilon:  0.93
episode:  111  score:  -189.74  avg_score:  -174.11  frames:  72  epsilon:  0.92
episode:  112  score:  -140.07  avg_score:  -172.19  frames:  94  epsilon:  0.92
episode:  113  score:  -304.20  avg_score:  -173.17  frames:  71  epsilon:  0.92
episode:  114  score:  -128.94  avg_score:  -173.02  frames:  72  epsilon:  0.92
episode:  115  score:  -294.

episode:  204  score:  -298.51  avg_score:  -150.87  frames:  97  epsilon:  0.87
episode:  205  score:  -77.35  avg_score:  -149.64  frames:  146  epsilon:  0.87
episode:  206  score:  -124.22  avg_score:  -151.21  frames:  68  epsilon:  0.87
episode:  207  score:  -40.81  avg_score:  -149.01  frames:  144  epsilon:  0.86
episode:  208  score:  -83.08  avg_score:  -148.73  frames:  67  epsilon:  0.86
episode:  209  score:  -268.33  avg_score:  -148.89  frames:  92  epsilon:  0.86
episode:  210  score:  -124.07  avg_score:  -148.88  frames:  105  epsilon:  0.86
episode:  211  score:  -123.48  avg_score:  -148.21  frames:  122  epsilon:  0.86
episode:  212  score:  -143.46  avg_score:  -148.25  frames:  80  epsilon:  0.86
episode:  213  score:  -86.07  avg_score:  -146.07  frames:  96  epsilon:  0.86
episode:  214  score:  -86.35  avg_score:  -145.64  frames:  94  epsilon:  0.86
episode:  215  score:  -55.86  avg_score:  -143.25  frames:  93  epsilon:  0.86
episode:  216  score:  -58.61 